In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Desenvolvimento do Modelo de Previsão de Preços de Ações\n",
    "\n",
    "Este notebook documenta o processo de desenvolvimento do modelo preditivo para previsão de preços de ações da Apple (AAPL) utilizando uma rede neural LSTM (Long Short-Term Memory).\n",
    "\n",
    "## Índice\n",
    "1. Importando bibliotecas\n",
    "2. Coleta de dados\n",
    "3. Análise exploratória dos dados\n",
    "4. Pré-processamento dos dados\n",
    "5. Construção do modelo LSTM\n",
    "6. Treinamento do modelo\n",
    "7. Avaliação do modelo\n",
    "8. Previsões e visualização\n",
    "9. Serialização do modelo\n",
    "10. Conclusões"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 1. Importando bibliotecas necessárias\n",
    "import os\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from datetime import datetime, timedelta\n",
    "\n",
    "import yfinance as yf\n",
    "from sklearn.preprocessing import MinMaxScaler\n",
    "from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score\n",
    "import tensorflow as tf\n",
    "from tensorflow.keras.models import Sequential\n",
    "from tensorflow.keras.layers import Dense, LSTM, Dropout\n",
    "from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint\n",
    "import joblib\n",
    "\n",
    "# Configurações do matplotlib para melhor visualização\n",
    "plt.style.use('seaborn')\n",
    "plt.rcParams['figure.figsize'] = (12, 6)\n",
    "plt.rcParams['font.size'] = 14\n",
    "\n",
    "# Verificar a versão do TensorFlow\n",
    "print(f\"TensorFlow version: {tf.__version__}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Coleta de dados\n",
    "\n",
    "Vamos coletar dados históricos da Apple (AAPL) utilizando a biblioteca yfinance."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Definir parâmetros\n",
    "symbol = \"AAPL\"  # Apple Inc.\n",
    "start_date = \"2018-01-01\"\n",
    "end_date = datetime.now().strftime(\"%Y-%m-%d\")\n",
    "\n",
    "# Baixar dados históricos\n",
    "stock_data = yf.download(symbol, start=start_date, end=end_date)\n",
    "\n",
    "# Exibir as primeiras linhas\n",
    "print(f\"Dados coletados: {len(stock_data)} registros\")\n",
    "stock_data.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Análise exploratória dos dados\n",
    "\n",
    "Vamos analisar os dados coletados para entender melhor o comportamento do preço das ações."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Verificar estatísticas descritivas\n",
    "stock_data.describe()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Verificar valores nulos\n",
    "print(\"Valores nulos por coluna:\")\n",
    "print(stock_data.isNone().sum())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualizar a evolução do preço de fechamento\n",
    "plt.figure(figsize=(14, 7))\n",
    "plt.plot(stock_data.index, stock_data['Close'], label='Preço de Fechamento')\n",
    "plt.title(f'Preço de Fechamento das Ações da {symbol}')\n",
    "plt.xlabel('Data')\n",
    "plt.ylabel('Preço ($)')\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Análise da distribuição do volume de negociações\n",
    "plt.figure(figsize=(14, 7))\n",
    "plt.plot(stock_data.index, stock_data['Volume'], label='Volume')\n",
    "plt.title(f'Volume de Negociações da {symbol}')\n",
    "plt.xlabel('Data')\n",
    "plt.ylabel('Volume')\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Verificar a distribuição dos retornos diários\n",
    "stock_data['Return'] = stock_data['Close'].pct_change() * 100\n",
    "\n",
    "plt.figure(figsize=(14, 7))\n",
    "plt.subplot(1, 2, 1)\n",
    "plt.plot(stock_data.index, stock_data['Return'], label='Retorno Diário')\n",
    "plt.title(f'Retornos Diários da {symbol}')\n",
    "plt.xlabel('Data')\n",
    "plt.ylabel('Retorno (%)')\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "sns.histplot(stock_data['Return'].dropna(), kde=True)\n",
    "plt.title('Distribuição dos Retornos Diários')\n",
    "plt.xlabel('Retorno (%)')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Verificar correlações entre as variáveis\n",
    "plt.figure(figsize=(10, 8))\n",
    "correlation_matrix = stock_data.corr()\n",
    "sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)\n",
    "plt.title(f'Matriz de Correlação para {symbol}')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Pré-processamento dos dados\n",
    "\n",
    "Agora vamos preparar os dados para o treinamento do modelo LSTM. Isso inclui:\n",
    "1. Selecionar apenas o preço de fechamento\n",
    "2. Normalizar os dados\n",
    "3. Criar sequências de dados para o treinamento do modelo"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Focar apenas nos preços de fechamento\n",
    "close_prices = stock_data['Close'].values.reshape(-1, 1)\n",
    "\n",
    "# Normalizar os dados para o intervalo [0, 1]\n",
    "scaler = MinMaxScaler(feature_range=(0, 1))\n",
    "scaled_data = scaler.fit_transform(close_prices)\n",
    "\n",
    "print(f\"Preço mínimo: ${close_prices.min()[0]:.2f}\")\n",
    "print(f\"Preço máximo: ${close_prices.max()[0]:.2f}\")\n",
    "print(f\"Dados normalizados no intervalo: [{scaled_data.min()}, {scaled_data.max()}]\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Definir o tamanho da janela de tempo (número de dias passados para usar como features)\n",
    "feature_window = 60  # 60 dias passados\n",
    "\n",
    "# Função para criar sequências para o modelo LSTM\n",
    "def create_sequences(data, window_size):\n",
    "    X, y = [], []\n",
    "    for i in range(len(data) - window_size):\n",
    "        X.append(data[i:i + window_size])\n",
    "        y.append(data[i + window_size])\n",
    "    return np.array(X), np.array(y)\n",
    "\n",
    "# Criar sequências para o modelo\n",
    "X, y = create_sequences(scaled_data, feature_window)\n",
    "\n",
    "print(f\"Formato de X: {X.shape}\")\n",
    "print(f\"Formato de y: {y.shape}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Dividir os dados em conjuntos de treino e teste (80% treino, 20% teste)\n",
    "train_size = int(len(X) * 0.8)\n",
    "X_train, X_test = X[:train_size], X[train_size:]\n",
    "y_train, y_test = y[:train_size], y[train_size:]\n",
    "\n",
    "print(f\"Conjunto de treino: {X_train.shape[0]} amostras\")\n",
    "print(f\"Conjunto de teste: {X_test.shape[0]} amostras\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Construção do modelo LSTM\n",
    "\n",
    "Vamos construir uma rede neural LSTM (Long Short-Term Memory) para previsão de séries temporais. Este tipo de rede é ideal para lidar com dados sequenciais como preços de ações."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Construir o modelo LSTM\n",
    "def build_model(input_shape):\n",
    "    model = Sequential([\n",
    "        LSTM(units=50, return_sequences=True, input_shape=input_shape),\n",
    "        Dropout(0.2),\n",
    "        LSTM(units=50, return_sequences=False),\n",
    "        Dropout(0.2),\n",
    "        Dense(units=25),\n",
    "        Dense(units=1)\n",
    "    ])\n",
    "    \n",
    "    model.compile(optimizer='adam', loss='mean_squared_error')\n",
    "    return model\n",
    "\n",
    "# Obter a forma dos dados de entrada\n",
    "input_shape = (X_train.shape[1], X_train.shape[2])\n",
    "print(f\"Formato de entrada: {input_shape}\")\n",
    "\n",
    "# Construir o modelo\n",
    "model = build_model(input_shape)\n",
    "\n",
    "# Resumo do modelo\n",
    "model.summary()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Treinamento do modelo\n",
    "\n",
    "Agora vamos treinar o modelo LSTM com os dados de treinamento. Usaremos callbacks para interromper o treinamento quando o modelo parar de melhorar e para salvar o melhor modelo."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Configurar callbacks\n",
    "early_stopping = EarlyStopping(\n",
    "    monitor='val_loss',\n",
    "    patience=10,\n",
    "    restore_best_weights=True\n",
    ")\n",
    "\n",
    "# Criar diretório para salvar o modelo se não existir\n",
    "os.makedirs('../models', exist_ok=True)\n",
    "model_checkpoint = ModelCheckpoint(\n",
    "    filepath='../models/best_model.h5',\n",
    "    monitor='val_loss',\n",
    "    save_best_only=True\n",
    ")\n",
    "\n",
    "# Treinar o modelo\n",
    "history = model.fit(\n",
    "    X_train, y_train,\n",
    "    batch_size=32,\n",
    "    epochs=100,\n",
    "    validation_data=(X_test, y_test),\n",
    "    callbacks=[early_stopping, model_checkpoint],\n",
    "    verbose=1\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualizar a curva de aprendizado\n",
    "plt.figure(figsize=(14, 7))\n",
    "plt.plot(history.history['loss'], label='Perda no Treino')\n",
    "plt.plot(history.history['val_loss'], label='Perda na Validação')\n",
    "plt.title('Curva de Aprendizado do Modelo')\n",
    "plt.xlabel('Épocas')\n",
    "plt.ylabel('Perda (MSE)')\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Avaliação do modelo\n",
    "\n",
    "Vamos avaliar o desempenho do modelo no conjunto de teste usando diversas métricas."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Fazer previsões no conjunto de teste\n",
    "y_pred = model.predict(X_test)\n",
    "\n",
    "# Desnormalizar os valores para a escala original\n",
    "y_test_real = scaler.inverse_transform(y_test)\n",
    "y_pred_real = scaler.inverse_transform(y_pred)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calcular métricas de avaliação\n",
    "mse = mean_squared_error(y_test_real, y_pred_real)\n",
    "rmse = np.sqrt(mse)\n",
    "mae = mean_absolute_error(y_test_real, y_pred_real)\n",
    "r2 = r2_score(y_test_real, y_pred_real)\n",
    "mape = np.mean(np.abs((y_test_real - y_pred_real) / y_test_real)) * 100\n",
    "\n",
    "print(f\"Erro Quadrático Médio (MSE): {mse:.4f}\")\n",
    "print(f\"Raiz do Erro Quadrático Médio (RMSE): {rmse:.4f}\")\n",
    "print(f\"Erro Absoluto Médio (MAE): {mae:.4f}\")\n",
    "print(f\"Coeficiente de Determinação (R²): {r2:.4f}\")\n",
    "print(f\"Erro Percentual Absoluto Médio (MAPE): {mape:.4f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Previsões e visualização\n",
    "\n",
    "Vamos visualizar as previsões do modelo comparadas aos valores reais."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Obter as datas do conjunto de teste\n",
    "train_size_with_window = train_size + feature_window\n",
    "test_dates = stock_data.index[train_size_with_window:train_size_with_window + len(y_test_real)]\n",
    "\n",
    "# Visualizar os resultados\n",
    "plt.figure(figsize=(16, 8))\n",
    "plt.plot(test_dates, y_test_real, label='Valor Real', color='blue')\n",
    "plt.plot(test_dates, y_pred_real, label='Previsão', color='red', linestyle='--')\n",
    "plt.title(f'Previsão vs. Valor Real - Preço de Fechamento da {symbol}')\n",
    "plt.xlabel('Data')\n",
    "plt.ylabel('Preço ($)')\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "plt.xticks(rotation=45)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualizar o erro de previsão\n",
    "plt.figure(figsize=(16, 8))\n",
    "\n",
    "# Calcular o erro\n",
    "prediction_error = y_test_real - y_pred_real\n",
    "plt.plot(test_dates, prediction_error, label='Erro de Previsão', color='green')\n",
    "plt.axhline(y=0, color='r', linestyle='-')\n",
    "plt.title(f'Erro de Previsão - {symbol}')\n",
    "plt.xlabel('Data')\n",
    "plt.ylabel('Erro ($)')\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "plt.xticks(rotation=45)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. Previsão para dias futuros\n",
    "\n",
    "Vamos usar o modelo para prever os preços para os próximos dias."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Definir o número de dias para prever\n",
    "prediction_days = 10\n",
    "\n",
    "# Obter a última sequência de dados\n",
    "last_window = scaled_data[-feature_window:].reshape(1, feature_window, 1)\n",
    "\n",
    "# Fazer previsões para os próximos 'prediction_days' dias\n",
    "predicted_prices = []\n",
    "current_window = last_window.copy()\n",
    "\n",
    "for _ in range(prediction_days):\n",
    "    # Fazer previsão para o próximo dia\n",
    "    next_pred = model.predict(current_window)\n",
    "    predicted_prices.append(next_pred[0, 0])\n",
    "    \n",
    "    # Atualizar a janela para incluir a previsão feita\n",
    "    current_window = np.append(current_window[:, 1:, :], \n",
    "                              np.array([[[next_pred[0, 0]]]]), \n",
    "                              axis=1)\n",
    "\n",
    "# Desnormalizar as previsões\n",
    "predicted_prices_real = scaler.inverse_transform(\n",
    "    np.array(predicted_prices).reshape(-1, 1)\n",
    ").flatten()\n",
    "\n",
    "# Criar datas para as previsões futuras\n",
    "last_date = stock_data.index[-1]\n",
    "future_dates = [last_date + timedelta(days=i+1) for i in range(prediction_days)]\n",
    "\n",
    "# Exibir as previsões\n",
    "for date, price in zip(future_dates, predicted_prices_real):\n",
    "    print(f\"{date.strftime('%Y-%m-%d')}: ${price:.2f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualizar as previsões futuras junto com os últimos dados históricos\n",
    "plt.figure(figsize=(16, 8))\n",
    "\n",
    "# Plotar os últimos 120 dias de dados históricos\n",
    "recent_dates = stock_data.index[-120:]\n",
    "recent_prices = stock_data['Close'].values[-120:]\n",
    "plt.plot(recent_dates, recent_prices, label='Dados Históricos', color='blue')\n",
    "\n",
    "# Plotar previsões futuras\n",
    "plt.plot(future_dates, predicted_prices_real, label='Previsões Futuras', color='red', marker='o', linestyle='--')\n",
    "\n",
    "# Adicionar linha vertical para separar dados históricos e previsões\n",
    "plt.axvline(x=last_date, color='green', linestyle='--', label='Hoje')\n",
    "\n",
    "plt.title(f'Previsões Futuras para {symbol}')\n",
    "plt.xlabel('Data')\n",
    "plt.ylabel('Preço ($)')\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "plt.xticks(rotation=45)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 10. Serialização do modelo\n",
    "\n",
    "Vamos salvar o modelo e o scaler para uso posterior na API."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Salvar o modelo treinado\n",
    "model_path = f\"../models/{symbol}_model.h5\"\n",
    "model.save(model_path)\n",
    "print(f\"Modelo salvo em {model_path}\")\n",
    "\n",
    "# Salvar o scaler\n",
    "scaler_path = \"../models/scaler.pkl\"\n",
    "joblib.dump(scaler, scaler_path)\n",
    "print(f\"Scaler salvo em {scaler_path}\")\n",
    "\n",
    "# Salvar a última janela para uso na API\n",
    "last_window_path = \"../models/last_window.npy\"\n",
    "np.save(last_window_path, last_window)\n",
    "print(f\"Última janela salva em {last_window_path}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 11. Conclusões\n",
    "\n",
    "Neste notebook, desenvolvemos um modelo LSTM para prever os preços de fechamento das ações da Apple (AAPL). O modelo apresentou um bom desempenho no conjunto de teste, com métricas satisfatórias.\n",
    "\n",
    "### Resultados\n",
    "- RMSE: Valor da raiz do erro quadrático médio\n",
    "- MAE: Valor do erro absoluto médio\n",
    "- MAPE: Percentual de erro médio\n",
    "- R²: Coeficiente de determinação\n",
    "\n",
    "### Próximos passos\n",
    "1. Implementar a API para servir previsões\n",
    "2. Configurar o monitoramento do modelo em produção\n",
    "3. Automatizar o processo de retreinamento quando necessário\n",
    "4. Considerar a inclusão de features adicionais para melhorar a precisão (como indicadores técnicos, dados macroeconômicos, etc.)\n",
    "\n",
    "O modelo deve ser usado com cautela para decisões de investimento, pois previsões de mercado estão sujeitas a muitos fatores imprevisíveis. Este é apenas um modelo preditivo baseado em padrões históricos."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}